 # Opis zadania
 W tym notatniku stworzymy system analityczno-raportowy, mający wspomagać przyszłe decyzje biznesowe. Dzięki przeprowadzonej wcześniej analizie eksploracyjnej danych, w trakcie spotkania prezentującego wykonaną pracę postanowiono w pierwszym kroku stworzyć następujące wizualizacje danych:
 - Wyświetlenie TOP 10 tras samolotów pod względem najniższego odsetka opóźnień w formie tabeli,
 - Wyświetlenie porównania roku 2019 oraz 2020 w formie wykresu słupkowego:
     - miesiąc do miesiąca,
     - dzień tygodnia do dnia tygodnia,
 - Wyświetlenie danych dzień po dniu w formie szeregu czasowego.

Raport powinien składać się z trzech stron - `TOP report`, `Comparision` i `Day by day reliability`.

Pracując na podstawie wcześniej zdefiniowanych widoków, nie musimy się tutaj martwić o przetwarzanie oraz procesowanie danych. Co więcej, wszystkie niuanse techniczne w postaci liczby wierszy, wykonywanych złączeń, filtracji są przeniesione na bazę danych. Z technicznego punktu widzenia, pomiędzy serwisem a bazą danych dochodzi do przesyłania mniejszej ilości danych.

Dopuszczamy jednak pewne aktualizacje co do ich struktury przykładowo poprzez _pivotowanie_, czyli obranie i doprowadzanie do postaci tej znanej z tabel przestawnych.

> **Uwaga:**  
> Przy pracy nad poprawkami w dashboardzie, pamiętaj, że aby odświeżyć stronę po wprowadzonych zmianach, należy **całość** uruchomić ponownie.

W tej części projektu końcowego wcielasz się ponownie w rolę BI Developera, który ma za zadanie stworzyć dashboard zgodny z wytycznymi biznesowymi dostarczonymi przez klienta. Pamiętaj, że osoba na tym stanowisku często ma kontakt z biznesem więc musi umieć przekazać informację o danych, które zawarła na dashboardzie by te zaangażowały odbiorców.


 Tutaj zaimportuj potrzebne biblioteki

In [1]:
import pandas as pd
import dash
from psycopg2 import connect
import plotly.graph_objects as go
import plotly.express as px
from dash import dash_table
from dash import dcc, html
from dash.dependencies import Input, Output, State

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [2]:
username = 'postgres'
password = 'Arch3olog!A'  

host = 'localhost'
database = 'airlines_data'
port = 5432

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [3]:
con = connect(database = database,
        user=username,
        password=password,
        host=host,
        port=port)

cursor = con.cursor()

 # Stworzenie tabeli dla `TOP 10 routes`
 W tym miejscu przygotujemy tabelkę oraz komponent, który zostanie następnie umieszczony w raporcie. Tabela powinna się składać z następujących kolumn:
 - Origin,
 - Destination,
 - Year,
 - Reliability,
 - Rank.

Wartości procentowe zaś powinny być postaci np. 13.87%.

Z tego względu konieczna będzie zmiana nazw kolumn dla wyników raportowania.

Wskazówki:
 - `Python - analiza danych > Dzień 5 - Pandas > Obróbka danych - część 2 > apply`,
 - [How to format percentage in python](https://stackoverflow.com/questions/5306756/how-to-print-a-percentage-value-in-python).

 Tutaj zaczytaj dane do ramki danych `top_routes_df` z widoku `reporting.top_reliability_roads`
 > Jeśli pojawi się komunikat `UserWarning: pandas only support SQLAlchemy`, z naszego punktu widzenia, możemy go śmiało zignorować.

In [4]:
cursor.execute('SELECT origin_airport_name, dest_airport_name, year, reliability, nb FROM reporting.top_reliability_roads')
columns = [desc[0] for desc in cursor.description]
rows = cursor.fetchall()

In [5]:
top_routes_df = pd.DataFrame(rows, columns = columns)

In [6]:
top_routes_df.head(3)

,origin_airport_name,dest_airport_name,year,reliability,nb
0,Chicago O'Hare International,LaGuardia,2019,0.27414023489696884450,1
1,LaGuardia,Chicago O'Hare International,2019,0.23896359923959726818,2
2,Los Angeles International,San Francisco International,2019,0.23410896708286038593,3


 Tutaj dokonaj aktualizacji formatu liczbowego dla kolumny `reliability`

In [7]:
top_routes_df['reliability'] = top_routes_df['reliability'].apply(lambda x: '{:.2f}%'.format(x * 100))
top_routes_df.head(3)


,origin_airport_name,dest_airport_name,year,reliability,nb
0,Chicago O'Hare International,LaGuardia,2019,27.41%,1
1,LaGuardia,Chicago O'Hare International,2019,23.90%,2
2,Los Angeles International,San Francisco International,2019,23.41%,3


 Tutaj odpowiednio dokonaj zmian nazewnictwa kolumn

In [8]:
top_routes_df = top_routes_df.rename(columns={'origin_airport_name': 'Origin', 'dest_airport_name': 'Destination', 'year': 'Year', 'reliability': 'Reliability', 'nb': 'Rank'})
top_routes_df.head(3)

,Origin,Destination,Year,Reliability,Rank
0,Chicago O'Hare International,LaGuardia,2019,27.41%,1
1,LaGuardia,Chicago O'Hare International,2019,23.90%,2
2,Los Angeles International,San Francisco International,2019,23.41%,3


 # Porównanie rok do roku (YoY) 2019 vs. 2020
 W tym miejscu stworzymy wykres oraz komponent, który zostanie następnie umieszczony w raporcie. Wykres powinien przedstawiać porównanie lat 2019 oraz 2020 po miesiącach względem zmiennej `reliability`.

 > Na wykresie chcemy również przedstawić dane, które nie mają porównania tj. od kwietnia do grudnia 2019.

W tym celu wykonamy poniższe zadania:
 - Przekształcimy tabelkę do formy tabeli przestawnej
 - Stworzymy wykres słupkowy porównujący dane rok do roku
 - Opakujemy stworzony wykres w komponent `Dash`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 1 > Budowanie wykresów plotly`,
 - `Python - analiza danych > Dzień 6 - Pandas c.d > Pivot`.

 Tutaj pobierz z widoku `reporting.year_to_year_comparision` dane do ramki `yoy_comparision_df`

In [9]:
cursor.execute('SELECT * FROM reporting.year_to_year_comparision')
columns2 = [desc[0] for desc in cursor.description]
rows2 = cursor.fetchall()
yoy_comparision_df = pd.DataFrame(rows2, columns = columns2)
yoy_comparision_df.head(3)


,year,month,flights_amount,reliability
0,2019,1,86826,0.18078686107847879667
1,2019,2,79514,0.21856528410091304676
2,2019,3,91789,0.17605595441719596030


In [10]:
yoy_comparision_df['reliability'] = yoy_comparision_df['reliability'].apply(lambda x: '{:.2f}%'.format(x * 100))

 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `yoy_comparision_to_plot_df`

In [11]:
yoy_comparision_to_plot_df = pd.pivot(yoy_comparision_df, index = 'month', columns= 'year',values=['reliability'])


In [12]:
yoy_comparision_to_plot_df 

reliability        
year         2019    2020
month                    
1          18.08%  12.58%
2          21.86%  13.44%
3          17.61%   9.28%
4          18.28%     NaN
5          20.47%     NaN
6          24.76%     NaN
7          21.67%     NaN
8          21.03%     NaN
9          14.77%     NaN
10         16.35%     NaN
11         13.92%     NaN
12         20.79%     NaN

 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `yoy_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [30]:
color_list = ["#bbe1c3","#a7cdbd","#869d7a","#8c8b6c","#91785d","#8e6b48","#8b5d33"]

In [25]:
yoy_comparision_fig = px.bar(
    data_frame=yoy_comparision_to_plot_df.stack().reset_index(),  # Konwersja do postaci, która jest obsługiwana przez Plotly
    x='month',
    y='reliability',
    color='year',
    color_continuous_scale=color_list,
    title="Year-on-year comparison of flight reliability",
    labels={"year": "Year", 'reliability': "Reliability (%)", 'month': "Month"},
    template = 'plotly_white',
)

yoy_comparision_fig.show()

C:\Users\ankaz\AppData\Local\Temp\ipykernel_33236\1754207909.py:2: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



 # Porównanie dzień tygodnia do dnia tygodnia (WKoWK) 2019 vs. 2020
 Wykonaj analogiczne do poprzedniego kroku, używając jednak w tym momencie danych z widoku `reporting.day_to_day_comparision`

 Tutaj pobierz z widoku `reporting.day_to_day_comparision` dane do ramki `day_to_day_comparision_df`

In [26]:
cursor.execute('SELECT * FROM reporting.day_to_day_comparision')
columns3 = [desc[0] for desc in cursor.description]
rows3 = cursor.fetchall()
day_to_day_comparision_df = pd.DataFrame(rows3, columns = columns3)
day_to_day_comparision_df.head(3)

,year,day_of_week,flights_amount
0,2019,1,164325
1,2019,2,162265
2,2019,3,160092


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `day_to_day_comparision_to_plot_df`

In [27]:
day_to_day_comparision_to_plot_df = pd.pivot(day_to_day_comparision_df, index = 'day_of_week', columns= 'year',values=['flights_amount'])

In [28]:
day_to_day_comparision_to_plot_df

flights_amount       
year                  2019   2020
day_of_week                      
1                   164325  37713
2                   162265  36396
3                   160092  37532
4                   162989  38472
5                   165283  38133
6                   127682  28415
7                   153106  34554

 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `day_to_day_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [34]:
day_to_day_comparision_fig = px.bar(
    data_frame= day_to_day_comparision_to_plot_df.stack().reset_index(),  # Konwersja do postaci, która jest obsługiwana przez Plotly
    x='day_of_week',
    y='flights_amount',
    color='year',
    color_continuous_scale=color_list,
    title="Comparison of flight reliability by day of the week",
    labels={"year": "Year", 'flights_amount': "Flights amount", 'day_of_week': "Day of week"},
    template = 'plotly_white',
)

day_to_day_comparision_fig.show()

C:\Users\ankaz\AppData\Local\Temp\ipykernel_33236\3015671375.py:2: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



 # Stworzenie szeregu czasowego
 W tym miejscu stworzymy wykres w formie szeregu czasowego, który zostanie umieszcony w raporcie. Wykres powinien przedstawiać dane w formie szeregu czasowego (dzień po dniu) dla lat 2019 oraz 2020. Dla możliwości identyfikacji poszczególnych lat, zostaliśmy dodatkowo poproszeni o nadanie innego koloru dla roku 2019 oraz 2020.

 > Pamiętaj o stworzeniu wykresu zgodnie z dobrymi praktykami.

 W tym miejscu pobierz dane do ramki `day_by_day_reliability_df`, z widoku `reporting.day_by_day_reliability`.

In [35]:
cursor.execute('SELECT * FROM reporting.day_by_day_reliability')
columns4 = [desc[0] for desc in cursor.description]
rows4 = cursor.fetchall()
day_by_day_reliability_df = pd.DataFrame(rows4, columns = columns4)
day_by_day_reliability_df.head(3)

,date,reliability
0,2019-01-01,0.18655589123867069486
1,2019-01-02,0.21510616080751827358
2,2019-01-03,0.14595170454545454545


In [36]:
day_by_day_reliability_df['reliability'] = day_by_day_reliability_df['reliability'].apply(lambda x: '{:.2f}%'.format(x * 100))
day_by_day_reliability_df.head(3)

,date,reliability
0,2019-01-01,18.66%
1,2019-01-02,21.51%
2,2019-01-03,14.60%


 Tutaj stwórz wykres liniowy na podstawie pobranych danych. Wynik zapisz do zmiennej `day_by_day_reliability_fig`.

In [37]:
day_by_day_reliability_df['date'] = pd.to_datetime(day_by_day_reliability_df['date'])

In [38]:
data_2019 = day_by_day_reliability_df[day_by_day_reliability_df['date'].dt.year == 2019]
data_2020 = day_by_day_reliability_df[day_by_day_reliability_df['date'].dt.year == 2020]

In [41]:
day_by_day_reliability_fig = px.line(title="Flight reliability in 2019 and 2020",
              labels={"date": "Date", "reliability": "Reliability (%)"},
              template = 'plotly_white',
)

day_by_day_reliability_fig.add_scatter(x=data_2019['date'], y=data_2019['reliability'], mode='lines', name='2019', line=dict(color='#bbe1c3'))
day_by_day_reliability_fig.add_scatter(x=data_2020['date'], y=data_2020['reliability'], mode='lines', name='2020', line=dict(color='#8b5d33'))

day_by_day_reliability_fig.show()

 # Stworzenie layoutów poszczególnych stron

 ## Stworzenie layout dla widoku `TOP 10 routes`
 W tym momencie przekonwertujemy zmienną `top_routes_df` w postać `Dash.DataTable`. Dodatkowo, dla czytelności, dodamy nagłówek opisujący co tabela przedstawia.

 Nagłówek wystylizuj używając poniższych wymagań:
 - tag: `H3`,
 - czcionka: `verdana`,
 - kolor: '#4444`,
 - tekst do wyświetlenia: `TOP 10 reliability routes in 2019 and 2020`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 3 > Dash - datatable`,
 - Dokmentacja metody `Pandas` - [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html).

 Tutaj przygotuj komponent `Dash` `DataTable`, zapisując wynik do zmiennej `top_routes_table`

In [42]:
top_routes_table = dash_table.DataTable(
    id = 'top routes table',
    columns= [{'name': 'Origin', 'id': 'Origin', 'type': 'text'},
        {'name': 'Destination', 'id': 'Destination', 'type': 'text'},
        {'name': 'Year', 'id': 'Year', 'type': 'numeric'},
        {'name': 'Reliability', 'id': 'Reliability', 'type': 'any'},
        {'name': 'Rank', 'id': 'Rank', 'type': 'numeric'}],
    data = top_routes_df.to_dict('records') )

 Tutaj przygotuj nagłówek dla tabeli, zapisz go pod nazwą `top_routes_page_title`

In [43]:
top_routes_page_title = html.H3(
    "TOP 10 reliability routes in 2019 and 2020",
    style = {"fontFamily": "verdana","color": "#444"},
)

 Tutaj przygotuj layout dla widoku `TOP 10 routes`. Konfigurację zapisz do zmiennej `top_routes_layout`

In [44]:
top_routes_layout = html.Div([
    top_routes_page_title,  # Nagłówek
    # Data table
    html.Div([
        top_routes_table  # DataTable
    ], id='top-routes-table-container'),  # Id dla kontenera DataTable
], id='top-routes-layout')  # Id dla całego layoutu

 ## Stworzenie layout dla widoku `Comparision`
 W tym momencie opakujemy zmienne `yoy_comparision_fig` oraz `day_to_day_comparision_fig` do komponentów `Dash`, a następnie stworzymy odpowiedni `layout` dla strony `comparision`. Wyniki zapiszemy do zmiennej `comparision_layout`.

 Tutaj opakuj `yoy_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `yoy_comparision_component`.

In [45]:
yoy_comparision_component = dcc.Graph(
    id='yoy-comparision-graph',  # Id dla komponentu wykresu
    figure=yoy_comparision_fig  # Przypisanie figury do komponentu
)

 Tutaj opakuj `day_to_day_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `day_to_day_comparision_component`.

In [46]:
day_to_day_comparision_component = dcc.Graph(
    id='day-to-day-comparision-graph',  # Id dla komponentu wykresu
    figure=day_to_day_comparision_fig  # Przypisanie figury do komponentu
)

 Tutaj stwórz odpowiedni `layout` dla widoku `comparision`. Wynik zapisz do zmiennej `comparision_layout`.

In [47]:
comparison_layout = html.Div(
    [   html.H3("Flights comparision",
            style = {"fontFamily": "verdana","color": "#444"},),
        # Komponent wykresu yoy_comparision_component
        yoy_comparision_component,
        # Komponent wykresu day_to_day_comparision_component
        day_to_day_comparision_component
    ],
    id="comparison-layout"  # Id dla całego layoutu
)

 Tutaj stwórz odpowiedni `layout` dla widoku `day by day`. Wynik zapisz do zmiennej `day_by_day_layout`.

In [48]:
day_by_day_component = dcc.Graph(
    id='day-by-day-reliability-graph',
    figure=day_by_day_reliability_fig
)

In [49]:
day_by_day_layout = html.Div([
    html.H3("Day by Day Reliability",
            style = {"fontFamily": "verdana","color": "#444"},),
    day_by_day_component
],
id = 'day-by-day-layout')

 # Konfigurowanie aplikacji
 Posiadamy już wszystkie niezbędne komponenty do stworzenia oraz uruchomienia aplikacji. Naszym zadaniem w tym miejscu będzie odpowiednie jej skonfigurowanie, aby obsługiwała więcej niż jedną stronę.

 Chcemy, aby możliwe było poruszanie się pomiędzy poniższymi stronami:
 - `TOP report` - strona domyślna,
 - `Comparision` - strona dostępna pod adresem: `comparision_reporting`,
 - `Day by day reporting` - strona dostępna pod adresem: `day_by_day_reporting`

Wskazówki:
 - `Wizualizacja danych > Dzień 4 > Callback context`,
 - `Wizualizacja danych > Dzień 4 > Aplikacja multipage`.
 - Używając komponentu [html.Button](https://community.plotly.com/t/button-with-link/11809), możesz łatwo stworzyć ładniejsze linki do poszczególnych stron.

 Tutaj przygotuj layout aplikacji

In [50]:
app = dash.Dash()

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div([
        html.A(html.Button('TOP report', id='top_report_button'), href='/top_routes_reporting'),
        html.A(html.Button('Comparision', id='comparision_button'), href='/comparision_reporting'),
        html.A(html.Button('Day by day reporting', id='day_by_day_button'), href='/day_by_day_reporting')
    ], style={'textAlign': 'center', 'margin': '20px'}),
    html.Div(id='page-content',
            children=html.Div(id='day_by_day_reporting')
            )
])

 Tutaj stwórz procedurę odpowiadającą za nawigację pomiędzy poszczególnymi stronami

In [51]:
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
     if  pathname == '/top_routes_reporting':
        return top_routes_layout
     elif pathname == '/comparision_reporting':
        return comparison_layout
     elif pathname == '/day_by_day_reporting':
        return day_by_day_layout
     else:
        # Domyślnie wyświetlany widok "TOP 10 reliability routes"
        return top_routes_layout

 Tutaj uruchom aplikację

In [52]:
app.run_server(debug=True)

 # Podsumowanie
 W tym notatniku stworzyliśmy zgodnie z wymaganiami raport do dyspozycji osób decyzyjnych. Kod źródłowy może w tym momencie zostać przekazany dalej, do działu IT, który następnie wdroży rozwiązanie na serwer dostępny dla każdej zainteresowanej osoby. W praktyce oznacza to koniec naszej pracy nad tym zadaniem. Choć warto dodać, że często po udostępnieniu raportu pojawiają się dodatkowe wymagania oraz komentarze ze względu na informacje, które są tam zawarte.

 W kolejnym notatniku podsumujemy sobie cały warsztat.